# 数据处理

## 带知识点label处理

In [1]:
import pandas as pd
grades = ['高中']
subjects = ['地理', '历史', '生物', '政治']
categories = {  '地理':['地球与地图','宇宙中的地球','生产活动与地域联系','人口与城市','区域可持续发展'],
                '历史':['古代史', '近代史', '现代史'],
                '生物':['现代生物技术专题','生物科学与社会','生物技术实践','稳态与环境','遗传与进化','分子与细胞'],
                '政治':['经济学常识', '科学思维常识', '生活中的法律常识','科学社会主义常识','公民道德与伦理常识','时事政治']
              }

df_target = pd.DataFrame(columns=['labels','item'])
for grade in grades:
    for subject in subjects:
        for category in categories[subject]:
            file = r'/mnt/d/Dataset/百度题库/'+grade+'_'+subject+'/origin/'+category+'.csv'
            df = pd.read_csv(file)
            print(f'{grade} {subject} {category} \tsize:{len(df)}')

            # 按网页顺序对其排序
            df['web-scraper-order'] = df['web-scraper-order'].apply(lambda x: int(x.split('-')[1]))
            df = df[['web-scraper-order','item']]
            df = df.sort_values(by='web-scraper-order')

            # 对文本处理
            def foo(x):
                x = "".join(x.split())
                # x = x[:x.index('题型')]
                return x

            # 删除文本中的换行
            df['item'] = df.item.apply(lambda x:  "".join(x.split()))
            df['labels'] = df.item.apply(lambda x: [grade, subject, category]+x[x.index('[知识点：]')+6:].split(',') if x.find('[知识点：]')!=-1 else [grade, subject, category])
            df['item'] = df.item.apply(lambda x:  x.replace('[题目]',''))
            df['item'] = df.item.apply(lambda x:  x[:x.index('题型')])

            df = df[['labels','item']]
            df_target = df_target.append(df)

len(df_target),df_target.head()

高中 地理 地球与地图 	size:431
高中 地理 宇宙中的地球 	size:3716
高中 地理 生产活动与地域联系 	size:1340
高中 地理 人口与城市 	size:1570
高中 地理 区域可持续发展 	size:130
高中 历史 古代史 	size:1000
高中 历史 近代史 	size:1640
高中 历史 现代史 	size:2330
高中 生物 现代生物技术专题 	size:1000
高中 生物 生物科学与社会 	size:3900
高中 生物 生物技术实践 	size:1770
高中 生物 稳态与环境 	size:3570
高中 生物 遗传与进化 	size:1040
高中 生物 分子与细胞 	size:2980
高中 政治 经济学常识 	size:566
高中 政治 科学思维常识 	size:260
高中 政治 生活中的法律常识 	size:170
高中 政治 科学社会主义常识 	size:573
高中 政治 公民道德与伦理常识 	size:1760
高中 政治 时事政治 	size:67


(29813,                                            labels  \
 163              [高中, 地理, 地球与地图, 等值线图, 地球运动的基本形式]   
 335                [高中, 地理, 地球与地图, 等值线图, 常见的天气系统]   
 378       [高中, 地理, 地球与地图, 大洋洲和南极洲, 地球仪，经纬网及其地理意义]   
 118  [高中, 地理, 地球与地图, 等高线地形图、地形剖面图, 地球仪，经纬网及其地理意义]   
 27   [高中, 地理, 地球与地图, 等高线地形图、地形剖面图, 地球仪，经纬网及其地理意义]   
 
                                                   item  
 163  下图是“地球自转等线速度示意图”，R、T在同一纬线上。据此回答下面小题。1.该区域所在的半球...  
 335  下图为“我国东南沿海某地区示意图”，关于此时A地的叙述，正确的是（）①雨水汇集，泛滥成灾②受...  
 378  中国第26次南极科考队经中山站(69°22′S,76°22′E)到昆仑站(80°25′S,7...  
 118  读下面经纬网图和等高线图，判断下列说法正确的是()A甲图全部在西半球，乙图在北半球B甲图比例...  
 27   茶树喜湿怕涝，海拔1500ｍ左右排水良好的山地缓坡，往往成为优质高山茶的重要产区，下图示意我...  )

In [2]:
from collections import Counter
import numpy as np

# 设置样本数量阈值
min_samples = 300
# 阈值 标签数
# 500   64
# 400   75
# 300   94
# 200   134
# 100   228

df = df_target.copy()
labels = []
for i in df.labels:
    labels.extend(i)
    
result = dict(sorted(dict(Counter(labels)).items(), key=lambda x:x[1], reverse=True))
lens = np.array(list(result.values()))
LABEL_NUM = len(lens[lens>min_samples])

# 选定数据label
label_target = set([k for k,v in result.items() if v > min_samples])

# 
df['labels'] = df.labels.apply(lambda x: x[:3] + list(set(x) - set(x[:3]) & label_target)) # 保证 grade subject category 在前三位置
df['labels'] = df.labels.apply(lambda x: None if len(x)<4 else x)                          # 去除没有知识点的数据
df = df[df.labels.notna()]

# 最终的labels数量
labels = []
[labels.extend(i) for i in df.labels]
LABEL_NUM = len(set(labels))

print(f'>{min_samples} datasize:{len(df)} multi_class:{LABEL_NUM}')

>300 datasize:22576 multi_class:95


处理成fasttext格式数据

In [3]:
# save
profix = '__label__'
    
if profix:
    df['labels'] = df.labels.apply(lambda x: [profix+i for i in x])

df['labels'] = df.labels.apply(lambda x: ' '.join(x))

# shuffle
df = df.sample(frac=1)

file = rf'/mnt/d/Dataset/百度题库/baidu_{LABEL_NUM}{profix}.txt'

with open(file, 'w') as f:
    for index, row in df.iterrows():
        f.write(row['labels'] + ' ' +row['item'] +'\n')
print('fasttext data file generated! ', file)

fasttext data file generated!  /mnt/d/Dataset/百度题库/baidu_95__label__.txt


处理成csv格式数据

In [297]:
# save
profix = ''
    
if profix:
    df['labels'] = df.labels.apply(lambda x: [profix+i for i in x])

df['labels'] = df.labels.apply(lambda x: ' '.join(x))

# shuffle
df = df.sample(frac=1)

file = rf'/mnt/d/Dataset/百度题库/baidu_{LABEL_NUM}{profix}.csv'

df.to_csv(file, index=False, sep=',', header=False, encoding='UTF8') # 当sep 字符在df中存在会在字符串前后添加引号
print('csv data file generated! ', file)

csv data file generated!  /mnt/d/Dataset/百度题库/baidu_95.csv


In [298]:
from tensorflow.keras import preprocessing
import jieba
padding_size = 200

df = pd.read_csv('/mnt/d/Dataset/百度题库/baidu_95.csv', header=None, names=["labels", "item"], dtype=str)
df['item'] = df.item.apply(lambda x: list(jieba.cut(x)))
corpus = df.item.tolist()
text_preprocesser = preprocessing.text.Tokenizer(num_words=5000)
text_preprocesser.fit_on_texts(corpus)
x = text_preprocesser.texts_to_sequences(corpus)
word_dict = text_preprocesser.word_index
# json.dump(word_dict, open(vocab_file, 'w'), ensure_ascii=False)
# max_doc_length = max([len(each_text) for each_text in x])
x = preprocessing.sequence.pad_sequences(x, maxlen=padding_size,
                                         padding='post', truncating='post')

# for y
print("Find words: {:d}".format(len(word_dict)))
print("Vocabulary size: {:d}".format(vocab_size))
print("Shape of train data: {}".format(np.shape(x)))

Find words: 50726
Vocabulary size: 50727
Shape of train data: (22576, 200)


In [299]:
from sklearn.preprocessing import MultiLabelBinarizer
y = df.labels.apply(lambda x: set(x.split())).tolist()
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)
mlb.__dict__

{'classes': None,
 'sparse_output': False,
 'classes_': array(['“重农抑商”政策', '不完全显性', '与细胞分裂有关的细胞器', '中央官制——三公九卿制', '中心体的结构和功能',
        '人体免疫系统在维持稳态中的作用', '人体水盐平衡调节', '人体的体温调节', '人口与城市', '人口增长与人口问题',
        '人口迁移与人口流动', '人工授精、试管婴儿等生殖技术', '伴性遗传', '体液免疫的概念和过程', '免疫系统的功能',
        '免疫系统的组成', '公民道德与伦理常识', '兴奋在神经元之间的传递', '兴奋在神经纤维上的传导', '内环境的稳态',
        '内质网的结构和功能', '农业区位因素', '减数分裂与有丝分裂的比较', '减数分裂的概念', '分子与细胞',
        '劳动就业与守法经营', '历史', '古代史', '器官移植', '地球与地图', '地球所处的宇宙环境',
        '地球的内部圈层结构及特点', '地球的外部圈层结构及特点', '地球运动的地理意义', '地球运动的基本形式', '地理',
        '垄断组织的出现', '培养基与无菌技术', '基因工程的原理及技术', '基因工程的概念', '基因的分离规律的实质及应用',
        '基因的自由组合规律的实质及应用', '复等位基因', '夏商两代的政治制度', '太阳对地球的影响', '宇宙中的地球',
        '工业区位因素', '拉马克的进化学说', '政治', '文艺的春天', '核糖体的结构和功能', '海峡两岸关系的发展',
        '液泡的结构和功能', '清末民主革命风潮', '溶酶体的结构和功能', '激素调节', '现代史', '现代生物技术专题',
        '生产活动与地域联系', '生命活动离不开细胞', '生态系统的营养结构', '生活中的法律常识', '生物', '生物工程技术',
        '生物性污染', '生物技术在其他方面的应用', '生物技术实践', '生物科学与社会', '皇帝制度',
        '社会主义市场经济的伦理要求', '社会主义

## 不带知识点数据处理

### 高中政治

In [4]:
import pandas as pd

labels = ['经济学常识', '科学思维常识', '生活中的法律常识','科学社会主义常识','公民道德与伦理常识','时事政治']
category = '时事政治'

file = r'/mnt/d/Dataset/百度题库/高中_政治/origin/'+category+'.csv'
df = pd.read_csv(file)
print('size:', len(df))

# 按网页顺序对其排序
df['web-scraper-order'] = df['web-scraper-order'].apply(lambda x: int(x.split('-')[1]))
df = df[['web-scraper-order','item']]
df = df.sort_values(by='web-scraper-order')

# 对文本处理
def foo(x):
    x = x.strip()
    x = x.replace('\n','')
    x = x.replace('\t','')
    x = x.replace('\r','')
    x = x.replace(' ','')
    x = x.replace(' ','')
    x = x[:x.index('题型')]
    return x

# 删除文本中的换行
df['item'] = df.item.apply(lambda x: foo(x))
df = df['item']

# save
with open(r'/mnt/d/Dataset/百度题库/高中_政治/'+category+'.csv','w',encoding='utf8') as f:
    f.write('\n'.join(list(df.values)))
category+'Done!'

size: 67


'时事政治Done!'

### 高中历史

In [17]:
import pandas as pd

labels = ['古代史', '近代史', '现代史']
category = labels[2]

file = r'/mnt/d/Dataset/百度题库/高中_历史/origin/'+category+'.csv'
df = pd.read_csv(file)
print('size:', len(df))

# 按网页顺序对其排序
df['web-scraper-order'] = df['web-scraper-order'].apply(lambda x: int(x.split('-')[1]))
df = df[['web-scraper-order','item']]
df = df.sort_values(by='web-scraper-order')

# 对文本处理
def foo(x):
    x = x.strip()
    x = x.replace('\n','')
    x = x.replace('\t','')
    x = x.replace('\r','')
    x = x.replace(' ','')
    x = x.replace(' ','')
    x = x[:x.index('题型')]
    return x

# 删除文本中的换行
df['item'] = df.item.apply(lambda x: foo(x))
# 删除带图数据
df = df[df.item.str.contains('图')==False]
df = df['item']

# save
with open(r'/mnt/d/Dataset/百度题库/高中_历史/'+category+'.csv','w',encoding='utf8') as f:
    f.write('\n'.join(list(df.values)))
category+'Done!'+str(len(df))

size: 2330


'现代史Done!2139'

### 高中-地理

In [11]:
import pandas as pd

labels = ['地球与地图','宇宙中的地球','生产活动与地域联系','人口与城市','区域可持续发展']
category = labels[0]

file = r'/mnt/d/Dataset/百度题库/高中_地理/origin/'+category+'.csv'
df = pd.read_csv(file)
print('size:', len(df))

# 按网页顺序对其排序
df['web-scraper-order'] = df['web-scraper-order'].apply(lambda x: int(x.split('-')[1]))
df = df[['web-scraper-order','item']]
df = df.sort_values(by='web-scraper-order')

# 对文本处理
def foo(x):
    x = x.strip()
    x = x.replace('\n','')
    x = x.replace('\t','')
    x = x.replace('\r','')
    x = x.replace(' ','')
    x = x.replace(' ','')
    x = x[:x.index('题型')]
    return x

# 删除文本中的换行
df['item'] = df.item.apply(lambda x: foo(x))
# 删除带图数据
df = df[df.item.str.contains('图')==False]
df = df['item']

# save
with open(r'/mnt/d/Dataset/百度题库/高中_地理/'+category+'.csv','w',encoding='utf8') as f:
    f.write('\n'.join(list(df.values)))
category+'Done!'+str(len(df))

size: 431


'地球与地图Done!78'

### 高中-生物

In [17]:
import pandas as pd

labels = ['现代生物技术专题','生物科学与社会','生物技术实践','稳态与环境','遗传与进化','分子与细胞']
category = labels[0]

file = r'/mnt/d/Dataset/百度题库/高中_生物/origin/'+category+'.csv'
df = pd.read_csv(file)
print('size:', len(df))

# 按网页顺序对其排序
df['web-scraper-order'] = df['web-scraper-order'].apply(lambda x: int(x.split('-')[1]))
df = df[['web-scraper-order','item']]
df = df.sort_values(by='web-scraper-order')

# 对文本处理
def foo(x):
    x = x.strip()
    x = x.replace('\n','')
    x = x.replace('\t','')
    x = x.replace('\r','')
    x = x.replace(' ','')
    x = x.replace(' ','')
    x = x[:x.index('题型')]
    return x

# 删除文本中的换行
df['item'] = df.item.apply(lambda x: foo(x))
# 删除带图数据
df = df[df.item.str.contains('图')==False]
df = df['item']

# save
with open(r'/mnt/d/Dataset/百度题库/高中_生物/'+category+'.csv','w',encoding='utf8') as f:
    f.write('\n'.join(list(df.values)))
category+'Done!'+str(len(df))

size: 1000


'现代生物技术专题Done!688'

# 数据生成

In [2]:
import os
import pandas as pd

path = r'/mnt/d/Dataset/百度题库/'
label_id = dict(zip(['高中_地理','高中_历史','高中_生物','高中_政治'],range(4)))
label_count = {}

x = []
y = []


for label in os.listdir(path):
    for doc in os.listdir(path+label):
        if not doc.endswith('csv'):
            continue
        
        sublabel = os.path.splitext(doc)[0]
        data = open(path+label+'/'+doc).readlines()
        data = list(map(lambda x: x[4:], data))
        
        if len(data)<500:continue
        
        x.extend(data)
        label_count[label+'_'+sublabel] = len(data)
        label_id[label+'_'+sublabel] = len(label_id)
        y.extend([[label_id[label], label_id[label+'_'+sublabel]]]*len(data))

# 去除内容中的制表符
x = [''.join(i.split()) for i in x]


len(x), len(y), label_id, label_count, sum(label_count.values())
with open('./data/kkb/x_19.txt', 'w') as f:
    f.write('\n'.join(x))
with open('./data/kkb/y_19.txt', 'w') as f:
    f.write(repr(y))
with open('label_id_19.json','w') as f:
    f.write(repr(label_id))

# Graph of words
not finished

In [1]:
import jieba
import numpy as np

def count_words(s):
    stop_words = ['$', '?', '_', '“', '”', '、', '。', '《', '》','，','（', '）', '的', '了', '是']
    tokenstr = []
    result = {}
    
    word2pos = {}
    pos2word = {}
    
    words = jieba.cut(s)
    
    i = 0 
    for word in words:
        if word in stop_words: continue
        tokenstr.append(word)
        result[word] = result.get(word, 0) + 1
        pos2word[i] = word
        
        indexs = word2pos.get(word, [])
        indexs.append(i)
        word2pos[word] = indexs
        
        i+=1
    
    result = dict(sorted(result.items(), key=lambda x: (x[1],x[0]), reverse=True))
    wordslist = list(result.keys())
    assert len(set(tokenstr)) == len(wordslist)
    return (wordslist, tokenstr, word2pos, pos2word)


def fill_table(TD_list, related_tables,target_width, qqueue):
    TD_list[0] = qqueue[0] # TD_list 长度为target_width 第一个位置对应此单词在wlist中的索引。0,1,2...
    count = 1

    while qqueue != [] and count < target_width:
        use_index = qqueue[0] # 单词索引
        del qqueue[0]
        use_list = related_tables[use_index]  #取出use_index单词对应的相关单词。
        len1 = len(use_list)   # 查看 i对应 的相关单词的个数。
        len2 = target_width - count 
        if len1 >= len2:   # 大体意思应该是查看单词i对应的相关单词个数如果满足 target_width就直接从相关单词按顺序取出来填充到TD_list中。
            TD_list[count:] = use_list[:len2]  
            assert len(TD_list) == target_width
            count = target_width
            break
        else:              # 如果不满足就有多少填多少。剩下的用 -1填充。
            TD_list[count:count + len1] = use_list
            assert len(TD_list) == target_width
            count += len1
            for next_id in use_list:
                qqueue.append(next_id)
    for i in range(count, target_width):
        TD_list[i] = -1
        
        
def reorder(table, word2pos, pos2word, wlist, word2id):
    sort_table = []
    topn, neighbor = np.array(table).shape
    for i in range(topn):
        tmp = []
        tmp += word2pos[wlist[table[i][0]]] # record each center word index
        length = len(tmp)                   # occurred times of center words
        t = []                              # t is use to related words index
        for j in range(1, neighbor):
            t += word2pos[wlist[table[i][j]]]
        index = np.random.randint(len(t), size = 20-length)
        t = np.array(t)
        t = list(t[index])
        tmp = tmp + t                       # conccat the index of center word and index of its related words
        tmp.sort()
        for j in range(len(tmp)):
            tmp[j] = word2id[pos2word[tmp[j]]] # convert index to word_id
            # tmp[j] = pos2word[tmp[j]]       # convert index to word
        sort_table.append(tmp)
    
    return np.array(sort_table)


def text2matrix(s, sliding_window=3, target_width=5):
    """
    
    """
    (wlist, tokenwords, word2pos, pos2word) = count_words(s)
    word2id = {k:v for k,v in zip(wlist, range(len(wlist)))}
    wordslist_length = len(wlist)
    
    AM_table = [[0 for i in range(wordslist_length)] for j in range(wordslist_length)]
    
    # generate occurred matrix with sliding_window
    for num in range(len(tokenwords)-sliding_window+1):
        for i in range(sliding_window-1):
            for j in range(i+1, sliding_window):
                AM_table[wlist.index(tokenwords[num + i])][wlist.index(tokenwords[num + j])] += 1
                AM_table[wlist.index(tokenwords[num + j])][wlist.index(tokenwords[num + i])] += 1
                
    related_tables = {}
    for i in range(wordslist_length):
        related_tables[i] = [[index, num] for index, num in enumerate(AM_table[i]) if num > 0 and index != i]
        related_tables[i].sort(key=lambda x: x[1], reverse=True)
        related_tables[i] = [element[0] for element in related_tables[i]]
    
    TD_table = [[-1 for i in range(target_width)] for j in range(wordslist_length)]
    for i in range(wordslist_length):
        fill_table(TD_table[i], related_tables, target_width, [i]) # fill TD table with -1
    
    # TD_table = reorder(TD_table, word2pos, pos2word, wlist, word2id)
    
    # convert id to words: arrange word matrix
    awm = []
    for row in TD_table:
        awm.append([pos2word[i] for i in row])
    return wlist, awm # ,TD_table

# way1(ERNIE): to_tsv single label for baidu.ernie
先试试8分类任务  
已跑通 效果acc:0.72

In [68]:
# shuffle
import numpy as np
np.random.seed(1)
indexs = list(range(len(x)))
np.random.shuffle(indexs)
x_,y_ = np.array(x)[indexs], np.array(y)[indexs]


train_x, train_y = x_[:-4000],y_[:-4000]
dev_x, dev_y = x_[-4000:-2000],y_[-4000:-2000]
test_x, test_y = x_[-2000:],y_[-2000:]

with open('./data/kkb/test.tsv','w',encoding='utf8') as f:
    f.write('label\ttext_a\n')
    for (a,b) in zip(test_x, test_y):
        a = ''.join(a.split())+'\n'
        f.write(str(b[1]-4)+'\t'+a)
        # f.write(repr(list(b))+'\t'+a)
        
with open('./data/kkb/dev.tsv','w',encoding='utf8') as f:
    f.write('label\ttext_a\n')
    for (a,b) in zip(dev_x, dev_y):
        a = ''.join(a.split())+'\n'
        f.write(str(b[1]-4)+'\t'+a)
        # f.write(repr(list(b))+'\t'+a)

with open('./data/kkb/train.tsv','w',encoding='utf8') as f:
    f.write('label\ttext_a\n')
    i = 0
    for (a,b) in zip(train_x, train_y):
        a = ''.join(a.split())+'\n'
        f.write(str(b[1]-4)+'\t'+a)
        # f.write(repr(list(b))+'\t'+a)

# way1(ERNIE):to_tsv multi labels
13分类数据  
so badly

In [71]:
# shuffle
import numpy as np
np.random.seed(1)
indexs = list(range(len(x)))
np.random.shuffle(indexs)
x_,y_ = np.array(x)[indexs], np.array(y)[indexs]


train_x, train_y = x_[:-4000],y_[:-4000]
dev_x, dev_y = x_[-4000:-2000],y_[-4000:-2000]
test_x, test_y = x_[-2000:],y_[-2000:]

with open('./data/kkb/test_multi.tsv','w',encoding='utf8') as f:
    f.write('label\ttext_a\n')
    for (a,b) in zip(test_x, test_y):
        a = ''.join(a.split())+'\n'
        f.write(str(list(b))+'\t'+a)
        # f.write(repr(list(b))+'\t'+a)
        
with open('./data/kkb/dev_multi.tsv','w',encoding='utf8') as f:
    f.write('label\ttext_a\n')
    for (a,b) in zip(dev_x, dev_y):
        a = ''.join(a.split())+'\n'
        f.write(str(list(b))+'\t'+a)
        # f.write(repr(list(b))+'\t'+a)

with open('./data/kkb/train_multi.tsv','w',encoding='utf8') as f:
    f.write('label\ttext_a\n')
    i = 0
    for (a,b) in zip(train_x, train_y):
        a = ''.join(a.split())+'\n'
        f.write(str(list(b))+'\t'+a)
        # f.write(repr(list(b))+'\t'+a)

# way2(BERT):
## bert_keras多标签分类示例

In [ ]:
#! -*- coding:utf-8 -*-

import json
import numpy as np
import pandas as pd
from random import choice
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
import re, os
import codecs


maxlen = 100
config_path = 'model/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = 'model/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = 'model/chinese_L-12_H-768_A-12/vocab.txt'


token_dict = {}

with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)


neg = pd.read_excel('./data/bert_keras/neg.xls', header=None)
pos = pd.read_excel('./data/bert_keras/pos.xls', header=None)

data = []

for d in neg[0]:
    data.append((d, 0))

for d in pos[0]:
    data.append((d, 1))


# 按照9:1的比例划分训练集和验证集
random_order = list(range(len(data)))
np.random.shuffle(random_order)
train_data = [data[j] for i, j in enumerate(random_order) if i % 10 != 0]
valid_data = [data[j] for i, j in enumerate(random_order) if i % 10 == 0]


def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])


class data_generator:
    def __init__(self, data, batch_size=32):
        self.data = data
        self.batch_size = batch_size
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            np.random.shuffle(idxs)
            X1, X2, Y = [], [], []
            for i in idxs:
                d = self.data[i]
                text = d[0][:maxlen]
                x1, x2 = tokenizer.encode(first=text)
                y = d[1]
                X1.append(x1)
                X2.append(x2)
                Y.append([y])
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    yield [X1, X2], Y
                    [X1, X2, Y] = [], [], []


from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam


bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

for l in bert_model.layers:
    l.trainable = True

x1_in = Input(shape=(None,))
x2_in = Input(shape=(None,))

x = bert_model([x1_in, x2_in])
x = Lambda(lambda x: x[:, 0])(x)
p = Dense(1, activation='sigmoid')(x)

model = Model([x1_in, x2_in], p)
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-5), # 用足够小的学习率
    metrics=['accuracy']
)
model.summary()


train_D = data_generator(train_data)
valid_D = data_generator(valid_data)

model.fit_generator(
    train_D.__iter__(),
    steps_per_epoch=len(train_D),
    epochs=5,
    validation_data=valid_D.__iter__(),
    validation_steps=len(valid_D)
)

## BERT
val_loss: 0.0527 - val_micro_f1: 0.9258 - val_macro_f1: 0.8505  
test: 0.05427, 0.9299, 0.8615

In [ ]:
#! -*- coding:utf-8 -*-

import json
import numpy as np
import pandas as pd
from random import choice
from keras_bert import load_trained_model_from_checkpoint, Tokenizer
import re, os
import codecs


from keras.layers import *
from keras.models import Model
import keras.backend as K
from keras.optimizers import Adam

import keras
keras.__version__

### bert parameters

In [ ]:
maxlen = 100
config_path = 'model/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = 'model/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = 'model/chinese_L-12_H-768_A-12/vocab.txt'


token_dict = {}

with codecs.open(dict_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)


class OurTokenizer(Tokenizer):
    def _tokenize(self, text):
        R = []
        for c in text:
            if c in self._token_dict:
                R.append(c)
            elif self._is_space(c):
                R.append('[unused1]') # space类用未经训练的[unused1]表示
            else:
                R.append('[UNK]') # 剩余的字符是[UNK]
        return R

tokenizer = OurTokenizer(token_dict)

# Arrange word matrix

In [ ]:
import jieba
import numpy as np

def count_words(s):
    stop_words = ['$', '?', '_', '“', '”', '、', '。', '《', '》','，','（', '）', '\n', '的', '了', '是']
    tokenstr = []
    result = {}
    
    word2pos = {}
    pos2word = {}
    
    words = jieba.cut(s)
    
    i = 0 
    for word in words:
        if word in stop_words: continue
        tokenstr.append(word)
        result[word] = result.get(word, 0) + 1
        pos2word[i] = word
        
        indexs = word2pos.get(word, [])
        indexs.append(i)
        word2pos[word] = indexs
        
        i+=1
    
    result = dict(sorted(result.items(), key=lambda x: (x[1],x[0]), reverse=True))
    wordslist = list(result.keys())
    assert len(set(tokenstr)) == len(wordslist)
    return (wordslist, tokenstr, word2pos, pos2word)


def fill_table(TD_list, related_tables,target_width, qqueue):
    TD_list[0] = qqueue[0] # TD_list 长度为target_width 第一个位置对应此单词在wlist中的索引。0,1,2...
    count = 1

    while qqueue != [] and count < target_width:
        use_index = qqueue[0] # 单词索引
        del qqueue[0]
        use_list = related_tables[use_index]  #取出use_index单词对应的相关单词。
        len1 = len(use_list)   # 查看 i对应 的相关单词的个数。
        len2 = target_width - count 
        if len1 >= len2:   # 大体意思应该是查看单词i对应的相关单词个数如果满足 target_width就直接从相关单词按顺序取出来填充到TD_list中。
            TD_list[count:] = use_list[:len2]  
            assert len(TD_list) == target_width
            count = target_width
            break
        else:              # 如果不满足就有多少填多少。剩下的用 -1填充。
            TD_list[count:count + len1] = use_list
            assert len(TD_list) == target_width
            count += len1
            for next_id in use_list:
                qqueue.append(next_id)
    for i in range(count, target_width):
        TD_list[i] = -1
        
        
def reorder(table, word2pos, pos2word, wlist, word2id):
    sort_table = []
    topn, neighbor = np.array(table).shape
    for i in range(topn):
        tmp = []
        tmp += word2pos[wlist[table[i][0]]] # record each center word index
        length = len(tmp)                   # occurred times of center words
        t = []                              # t is use to related words index
        for j in range(1, neighbor):
            t += word2pos[wlist[table[i][j]]]
        index = np.random.randint(len(t), size = 20-length)
        t = np.array(t)
        t = list(t[index])
        tmp = tmp + t                       # conccat the index of center word and index of its related words
        tmp.sort()
        for j in range(len(tmp)):
            tmp[j] = word2id[pos2word[tmp[j]]] # convert index to word_id
            # tmp[j] = pos2word[tmp[j]]       # convert index to word
        sort_table.append(tmp)
    
    return np.array(sort_table)


def text2matrix(s, sliding_window=3, target_width=5):
    """
    
    """
    (wlist, tokenwords, word2pos, pos2word) = count_words(s)
    word2id = {k:v for k,v in zip(wlist, range(len(wlist)))}
    wordslist_length = len(wlist)
    
    AM_table = [[0 for i in range(wordslist_length)] for j in range(wordslist_length)]
    
    # generate occurred matrix with sliding_window
    for num in range(len(tokenwords)-sliding_window+1):
        for i in range(sliding_window-1):
            for j in range(i+1, sliding_window):
                AM_table[wlist.index(tokenwords[num + i])][wlist.index(tokenwords[num + j])] += 1
                AM_table[wlist.index(tokenwords[num + j])][wlist.index(tokenwords[num + i])] += 1
                
    related_tables = {}
    for i in range(wordslist_length):
        related_tables[i] = [[index, num] for index, num in enumerate(AM_table[i]) if num > 0 and index != i]
        related_tables[i].sort(key=lambda x: x[1], reverse=True)
        related_tables[i] = [element[0] for element in related_tables[i]]
    
    TD_table = [[-1 for i in range(target_width)] for j in range(wordslist_length)]
    for i in range(wordslist_length):
        fill_table(TD_table[i], related_tables, target_width, [i]) # fill TD table with -1
    
    # TD_table = reorder(TD_table, word2pos, pos2word, wlist, word2id)
    
    # convert id to words: arrange word matrix
    awm = []
    for row in TD_table:
        awm.append([pos2word[i] for i in row])
    return wlist, awm # ,TD_table

### Load Data with multi labels(Baidu)

In [ ]:
# 类别
n_class = 19
# 是否处理数据不平衡
imbalance = True
fixed_nums = 900
# 是否使用Arrange word matrix
AWM = True


x,y = None, None
if n_class == 13:
    x = open('./data/kkb/x.txt','r',encoding='utf8').readlines()
    y = eval(open('./data/kkb/y.txtn_class', 'r',encoding='utf8').readlines()[0])
elif n_class == 19:
    x = open('./data/kkb/x_19.txt','r',encoding='utf8').readlines()
    y = eval(open('./data/kkb/y_19.txt','r',encoding='utf8').readlines()[0])

print('{} class task, data size:{},{}'.format(n_class,len(x),len(y)))


# 类别不平衡
if imbalance:
    ys = [str(i) for i in y]
    from collections import Counter
    counts = Counter(ys)
    for k,v in counts.items():
        print(k,':',v)

    df = pd.DataFrame({'x':x,'y':ys})
    new_x,new_y = [],[]
    for k, v in counts.items():
        x_ = np.random.choice(df[df.y == k].x, fixed_nums)
        y_ = [eval(k)]*900
        new_x.extend(x_)
        new_y.extend(y_)
    print('after deal data imbalance, data size:',len(new_x),len(new_y))
    x,y = new_x, new_y

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(range(n_class))
y = mlb.fit_transform(y)
y = [list(i) for i in y]

# arrage word matrix
x_awm = []
if AWM:
    for row in x[:10]:
        _, awm = text2matrix(row)
        x_awm.append(list(np.reshape(awm[:20],(-1))))
x = x_awm


data = list(zip(x,y))
np.random.shuffle(data)
train_data = data[:-4000]
val_data = data[-4000:-2000]
test_data = data[-2000:]

### Data generator

In [ ]:
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = max(L)
    return np.array([
        np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x for x in X
    ])


class data_generator:
    def __init__(self, data, batch_size=32, multi_labels=False):
        self.data = data
        self.batch_size = batch_size
        self.multi_labels = multi_labels
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1
    def __len__(self):
        return self.steps
    def __iter__(self):
        while True:
            idxs = list(range(len(self.data)))
            np.random.shuffle(idxs)
            X1, X2, Y = [], [], []
            for i in idxs:
                d = self.data[i]
                text = d[0][:maxlen]
                x1, x2 = tokenizer.encode(first=text)
                y = d[1]
                X1.append(x1)
                X2.append(x2)
                Y.append([y])
                if len(X1) == self.batch_size or i == idxs[-1]:
                    X1 = seq_padding(X1)
                    X2 = seq_padding(X2)
                    Y = seq_padding(Y)
                    if self.multi_labels: Y = Y.reshape(-1, np.shape(Y)[-1])
                    yield [X1, X2], Y
                    [X1, X2, Y] = [], [], []

### Metic

In [89]:
def micro_f1(y_true, y_pred):
    """F1 metric.
    
    Computes the micro_f1 and macro_f1, metrics for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=0)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)
    
    """Macro_F1 metric.
    """
    precision = true_positives/(predicted_positives+K.epsilon())
    recall = true_positives/(possible_positives+K.epsilon())
    macro_f1 = K.mean(2*precision*recall/(precision+recall+K.epsilon()))
        
    """Micro_F1 metric.
    """
    precision = K.sum(true_positives)/K.sum(predicted_positives)
    recall = K.sum(true_positives)/K.sum(possible_positives)
    micro_f1 = 2*precision*recall/(precision+recall+K.epsilon())
    return micro_f1

def macro_f1(y_true, y_pred):
    """F1 metric.
    
    Computes the micro_f1 and macro_f1, metrics for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)), axis=0)
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)), axis=0)
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)), axis=0)
    
    """Macro_F1 metric.
    """
    precision = true_positives/(predicted_positives+K.epsilon())
    recall = true_positives/(possible_positives+K.epsilon())
    macro_f1 = K.mean(2*precision*recall/(precision+recall+K.epsilon()))
        
    """Micro_F1 metric.
    """
    precision = K.sum(true_positives)/K.sum(predicted_positives)
    recall = K.sum(true_positives)/K.sum(possible_positives)
    micro_f1 = 2*precision*recall/(precision+recall+K.epsilon())
    return macro_f1

### Model

In [ ]:
bert_model = load_trained_model_from_checkpoint(config_path, checkpoint_path, seq_len=None)

for l in bert_model.layers:
    l.trainable = True

x1_in = Input(shape=(None,))
x2_in = Input(shape=(None,))

x = bert_model([x1_in, x2_in])
x = Lambda(lambda x: x[:, 0])(x)
p = Dense(13, activation='sigmoid')(x)

model = Model([x1_in, x2_in], p)
# val_metric = Metrics([val_x,val_y])
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(1e-5), # 用足够小的学习率
    metrics=[micro_f1,macro_f1]
)
model.summary()

In [ ]:
train_D = data_generator(train_data,multi_labels=True)
valid_D = data_generator(val_data,multi_labels=True)
test_D = data_generator(test_data,multi_labels=True)

model.fit_generator(
    train_D.__iter__(),
    steps_per_epoch=len(train_D),
    epochs=1,
    validation_data=valid_D.__iter__(),
    validation_steps=len(valid_D),
    # callbacks=[val_metric],
)

In [ ]:
test_D = data_generator(test_data,multi_labels=True)
model.evaluate_generator(test_D.__iter__(), len(test_D))

# 效果汇总

|数据集|模型|类别|Acc|Micro-F1|Macro-F1|备注|
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|Baidu|ERNIE|2|0.73|-|-|single classify|
|Baidu|BERT|13|-|0.9299|0.8615|multi_labels classify 13|
|Baidu|BERT|19|-|0.8996 |0.6797|multi_labels classify 19|
|Biadu|FastText|19|-|0.42|0.21|multi_labels classify 19(imbalance)|
|Baidu|GCN-BERT|19|-|0.90|0.78|multi_labels classify 19(balance)|
|Baidu|GCN-BERT|19|-|0.89|0.69|multi_labels classify 19(imbalance)|
|Baidu|FastText|95|-|0.421|0.234|epoch 1000, ngram 5, dim 50|
|Baidu|TextCnn|95|-|0.00478|0.028|epoch 10, lr 0.005, padding 128|
|Baidu|BERT|21|0.7958|-|-|BERT 4 labels result|
|Baidu|BERT|95|0.5788|0.917|0.781|only BERT|